In [13]:
import cv2

key = cv2.waitKey(1)
webcam = cv2.VideoCapture(0)

while True:
  try:
    check, fram e = webcam.read()
    print(check)
    print(frame)
    cv2.imshow("Capturing", frame)
    key = cv2.waitKey(1)
    if key == ord('s'):
      cv2.imwrite(filename = 'saved_img.jpg', img = frame)
      webcam.release()
      img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
      img_new = cv2.imshow('Captured Image', img_new)
      cv2.waitKey(1650)
      cv2.destroyAllWindows()
      print('Processing Image...')
      img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
      print("Converting RGB image to grayscale...")
      gray = cv2.cvtColor(img_, cv2.color_BGR2GRAY)
      print("resizing image to 28x28 scale...")
      img_ = cv2.resize(gray,(28,28))
      print("Resized...")
      img_resized = cv2.imwrite(filename='saved_img-final.jpg', img=img_)
      print("Image saved!")
      break

    elif key == ord('q'):
      print("Turning off camera.")
      webcam.release()
      print('Camera off.')
      print('Program ended.')
      cv2.destroyAllWindows()
      break

  except (KeyboardInterrupt):
    print("Turning off camera.")
    webcam.release()
    print("Camera off.")
    print("Program ended.")
    cv2.destroyAllWindows()
    break

True
[[[203 204 198]
  [203 204 198]
  [202 203 197]
  ...
  [ 28  24  46]
  [ 27  23  45]
  [ 26  22  44]]

 [[203 204 198]
  [203 204 198]
  [202 203 197]
  ...
  [ 28  24  45]
  [ 27  23  44]
  [ 26  22  43]]

 [[204 205 199]
  [203 204 198]
  [203 204 198]
  ...
  [ 28  25  45]
  [ 27  24  44]
  [ 26  23  43]]

 ...

 [[  3   1   3]
  [  3   1   3]
  [  3   1   3]
  ...
  [ 95  89  91]
  [ 94  88  90]
  [ 96  90  92]]

 [[  3   1   3]
  [  3   1   3]
  [  3   1   3]
  ...
  [ 98  91  93]
  [ 96  89  91]
  [ 99  92  94]]

 [[  3   1   3]
  [  3   1   3]
  [  3   1   3]
  ...
  [ 99  91  94]
  [ 98  90  93]
  [100  92  95]]]
True
[[[200 201 195]
  [200 201 195]
  [199 200 194]
  ...
  [ 26  25  38]
  [ 26  24  39]
  [ 25  23  39]]

 [[201 202 196]
  [200 201 195]
  [200 201 195]
  ...
  [ 26  25  38]
  [ 26  24  39]
  [ 25  23  39]]

 [[202 203 197]
  [201 202 196]
  [200 201 195]
  ...
  [ 26  25  38]
  [ 26  24  39]
  [ 25  23  39]]

 ...

 [[  3   1   3]
  [  3   1   3]
  [  3   1

Web Scraping


In [15]:
import requests as rq
from bs4 import BeautifulSoup as bs

def getData(url):
    r = rq.get(url)
    return r.text

htmldata = getData("https://www.google.com/")
soup = bs(htmldata, 'html.parser')
for item in soup.find_all('img'):
    print(item['src'])

/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png


In [ ]:
anotherData = getData('https://www.electronics-tutorial.net/analog-integrated-circuits/op-amp-comparators/non-inverting-comparator/1')
tite = bs(anotherData, 'html.parser')
print(tite.find_all('img'))

Using Selenium

In [26]:
import sys
from selenium import webdriver
import time
import requests
import shutil
import os
import getpass
import urllib.request
import io
import time
from PIL import Image

user = getpass.getuser()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
driver.get(search_url.format(q='Car'))

def scroll_to_end(driver):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(5)

def getImageUrls(name, totalImgs, driver):
    search_url =  "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
    driver.get(search_url.format(q=name))
    img_urls = set()
    img_count = 0
    results_start=0

    while(img_count<totalImgs):
        scroll_to_end(driver)
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults = len(thumbnail_results)
        print(f"Found: {totalResults} search results.Extracting links from{results_start}")

        for img in thumbnail_results[results_start:totalResults]:
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.add(actual_image.get_attribute('src'))

                img_count = len(img_urls)

                if img_count >= totalImgs:
                    print(f"Found: {img_count} image links")
                    break
                else:
                    print(f"Found: ", img_count, "looking for more links...")
                    load_more_button = driver.find_element_by_css_selector(".mye4qd")
                    driver.execute_script("document.querySelector('.mye4qd').click();")
                    results_start = len(thumbnail_results)
    return img_urls

def downloadImages(folder_path, file_name, url):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD{url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')

        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

def saveInDestFolder(searchNames, destDir, totalImgs, driver):
    for name in list(searchNames):
        path = os.path.join(destDir, name)
        if not os.path.isdir(path):
            os.mkdir(path)
        print('Current Path reached', path)
        totalLinks = getImageUrls(name, totalImgs, driver)
        print('total links', totalLinks)

    if totalLinks is None:
        print('images not found for: ', name)
    else:
        for i, link in enumerate(totalLinks):
            file_name = f"{i:150}.jpg"
            downloadImages(path, file_name, link)

searchNames = ['cat']
destDir=f'/content/drive/My Drive/Colab Notebooks/Dataset/'
totalImgs=5
saveInDestFolder(searchNames,destDir,totalImgs,driver)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Colab Notebooks/Dataset/cat'

Webscraping of movies information using beautifulsoup

In [40]:
from requests import get
url =   'https://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
agent = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 OPR/89.0.4389.90"}
response = get(url, headers=agent)
print(response.text[:500])

<!DOCTYPE html><html lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><script>window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
             


In [41]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
headers = {'Accept-Language' : 'en-US, en; q=0.8'}
type(html_soup)

bs4.BeautifulSoup

In [44]:
movie_containers =  html_soup.find_all('div', class_ = 'ipc-metadata-list-summary-item__c')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [45]:
first_movie = movie_containers[0]
first_movie

<div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t"></span><div class="sc-ab6fa25a-3 bVYfLY dli-parent"><div class="sc-ab6fa25a-2 gOsifL"><div class="sc-e5a25b0f-0 jQjDIb dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div aria-label="add to watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--s ipc-watchlist-ribbon--base ipc-watchlist-ribbon--loading ipc-watchlist-ribbon--onImage ipc-poster__watchlist-ribbon" role="button" tabindex="0"><svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg"><polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.2926049 24 31.7728343"></polygon><polygon class="ipc-watchlist-ribbon__bg-hover" poin

In [69]:
first_name = first_movie.h3.text
first_name[3:]

'Logan'

In [70]:
first_year = movie_containers[0].find('span', class_ = "sc-b0691f29-8 ilsLEX dli-title-metadata-item")
first_year

<span class="sc-b0691f29-8 ilsLEX dli-title-metadata-item">2017</span>

In [71]:
first_year = first_year.text
first_year

'2017'

The IMDB rating 

In [72]:
first_imdb = movie_containers[0].find('span', class_ = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
first_imdb.text[:3]

'8.1'

The Metascore

In [73]:
first_mscore = movie_containers[0].find('span', class_ = 'sc-b0901df4-0 bcQdDJ metacritic-score-box')
first_mscore = first_mscore.text
print(first_mscore)

77


In [75]:
first_votes = movie_containers[0].find('span', class_ = 'ipc-rating-star--voteCount')
first_votes.text[2:-1]

'827K'

The Script

In [81]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

for container in movie_containers:
    names.append(container.find('h3', class_ = "ipc-title__text").text[3:])
    years.append(container.find('span', class_ = "sc-b0691f29-8 ilsLEX dli-title-metadata-item"))
    imdb_ratings.append(container.find('span', class_ = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating").text[:3])
    metascores.append(container.find('span', class_ = 'sc-b0901df4-0 bcQdDJ metacritic-score-box' ))
    votes.append(container.find('span', class_ = 'ipc-rating-star--voteCount'))

print(len(names))
print(len(years))
print(len(imdb_ratings))
print(len(metascores))
print(len(votes))


50
50
50
50
50


In [82]:
import pandas as pd
test_df = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      50 non-null     object
 1   year       50 non-null     object
 2   imdb       50 non-null     object
 3   metascore  41 non-null     object
 4   votes      50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB
None


,movie,year,imdb,metascore,votes
0,Logan,[2017],8.1,[77],"[ (, , 827K, , )]"
1,Thor: Ragnarok,[2017],7.9,[74],"[ (, , 813K, , )]"
2,Guardians of the Galaxy Vol. 2,[2017],7.6,[67],"[ (, , 756K, , )]"
3,Dunkirk,[2017],7.8,[94],"[ (, , 736K, , )]"
4,Spider-Man: Homecoming,[2017],7.4,[73],"[ (, , 716K, , )]"
5,Wonder Woman,[2017],7.3,[76],"[ (, , 698K, , )]"
6,Get Out,[2017],7.8,[85],"[ (, , 691K, , )]"
7,Star Wars: Episode VIII - The Last Jedi,[2017],6.9,[84],"[ (, , 670K, , )]"
8,Blade Runner 2049,[2017],8.0,[81],"[ (, , 658K, , )]"
9,Baby Driver,[2017],7.5,[86],"[ (, , 605K, , )]"


In [ ]:
from time import time 
from time import sleep
from random import randint

from IPython.display import clear_output
pages = ['1', '2', '3', '4', '5']
years_url = ['2017', '2018', '2019', '2020']

names = []
years = []
imdb_ratings = []
metascores = []
votes = []
